#### Automating Column Naming Schemes from Reflected Tables

In the previous section `Naming Columns Distinctly from Attribute Names`, we showed how a `Column` __explicitly mapped__ to a class can have a _different attribute name_ than the column. But what if we aren't listing out `Column` objects _explicitly_, and instead are _automating_ the production of `Table` objects using __reflection__ (i.e. as described in `Reflecting Database Objects`)? In this case we can make use of the `DDLEvents.column_reflect()` event to __intercept the production of `Column` objects__ and provide them with the `Column.key` of our choice. The event is __most easily associated__ with the `MetaData` object that's in use, such as below we use the one linked to the `declarative_base` instance.

In [1]:
from sqlalchemy import Column, Integer, String, Table, event, create_engine
from sqlalchemy.orm import declarative_base

In [2]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
Base = declarative_base()

In [3]:
@event.listens_for(Base.metadata, "column_reflect")
def column_reflect(inspector, table, column_info):
    # set column.key = "attr_<lower_case_name>"
    column_info["key"] = f"attr_{column_info['name'].lower()}"

With the above _event_, the _reflection_ of `Column` objects will be __intercepted with our event__ that _adds_ a new `".key"` element, such as in a mapping as below.

In [4]:
class UserTable(Base):
    __tablename__ = "user"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)

In [5]:
Base.metadata.create_all(engine)

2022-11-25 15:50:48,735 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-25 15:50:48,736 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2022-11-25 15:50:48,737 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-25 15:50:48,739 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user")
2022-11-25 15:50:48,741 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-25 15:50:48,744 INFO sqlalchemy.engine.Engine 
CREATE TABLE user (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-11-25 15:50:48,746 INFO sqlalchemy.engine.Engine [no key 0.00154s] ()
2022-11-25 15:50:48,748 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
class User(Base):
    __table__ = Table(
        "user",
        Base.metadata,
        autoload_with=engine,
    )

The approach also works with both the `DeferredReflection` base class as well as with the `Automap extension`. _For automap specifically_, see the section `Intercepting Column Definitions` for background.